## Imports and settings

In [1]:
import pandas as pd
import csv
import plotly.express as px
import plotly.graph_objects as go
from src.scraper import get_range
from datetime import date
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 5000

## Read data

In [2]:
# df = pd.read_pickle('data/reports.pkl')
df = get_range(date(2018, 8, 1), date(2019, 6, 1))

##  Filter reports by keywords

In [3]:
def keyword_search(df, keywords):
    filtered_df = pd.DataFrame()
    for index, row in df.iterrows():
        for word in keywords:
            if word in row['Incident'].lower() or word in row['Narrative'].lower():
                filtered_df = filtered_df.append(row)
    return filtered_df

alcohol_keywords = {'alcohol', 'drinking', 'intoxicated'}
drug_keywords = {'marijuana', 'drug', 'paraphernalia'}

alcohol_df = keyword_search(serious_df, alcohol_keywords)
drug_df = keyword_search(serious_df, drug_keywords)

## Filter reports to only residence halls

In [4]:
halls = pd.read_csv('data/hall_populations.csv', sep='\t')

def get_residence_halls(df):
    filtered_df = pd.DataFrame()
    for index, row in df.iterrows():
        for hall in halls['Location']:
            if hall in row['Location'] and ' Rd' not in row['Location'] and ' Dr' not in row['Location']:
                new_row = row.copy()
                new_row['Location'] = hall
                filtered_df = filtered_df.append(new_row)
                continue
    return filtered_df

alcohol_halls = get_residence_halls(alcohol_df)
drug_halls = get_residence_halls(drug_df)

## Get stats for each residence hall

In [5]:
def get_stats_by_halls(df):
    sf = df['Location'].value_counts()
    halls_stats = pd.DataFrame({'Location':sf.index, 'Incident sum':sf.values})
    halls_stats.insert(2, 'Population', 0)
    halls_stats.insert(3, 'Incidents per 100 students', 0.0)

    for index, row in halls_stats.iterrows():
        hall_population = halls.loc[halls['Location'] == row['Location']]['Population'].values[0]
        halls_stats.at[index, 'Population'] = hall_population
        halls_stats.at[index, 'Incidents per 100 students'] = (row['Incident sum'] / hall_population) * 100

    for index, row in halls.iterrows():
        if row['Location'] not in halls_stats['Location'].values:
            halls_stats.loc[-1] = [row['Location'], 0, row['Population'], 0.0]
            halls_stats.index = halls_stats.index + 1
            halls_stats = halls_stats.sort_index()
    return halls_stats

alcohol_halls_stats = get_stats_by_halls(alcohol_halls)
drug_halls_stats = get_stats_by_halls(drug_halls)

## Draw graphs

In [8]:
drug_halls_stats = drug_halls_stats.sort_values(by=['Incident sum'], ascending=False, ignore_index=True)
fig = go.Figure(data=[
    go.Bar(name='Drug related incidents', x=drug_halls_stats['Location'], y=drug_halls_stats['Incident sum'], marker_color='#548f6f'),
    go.Bar(name='Alcohol related incidents', x=alcohol_halls_stats['Location'], y=alcohol_halls_stats['Incident sum'],      marker_color='#f28e1c')
])
fig.update_layout(
    title={
        'text': "Number of Drug & Alcohol Related Incidents by Residence Hall for the 2018-19 school year",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    font=dict(
        family="Helvetica",
        size=24
    ),
    xaxis_title='Residence Hall',
    yaxis_title='Total incidents reported',
    barmode='group',
    width=1920,
    height=1080,
    legend=dict(
        yanchor="top",
        y=0.96,
        xanchor="left",
        x=0.8
))
fig.write_image("images/fig3.png")

drug_halls_stats = drug_halls_stats.sort_values(by=['Incidents per 100 students'], ascending=False, ignore_index=True)
fig = go.Figure(data=[
    go.Bar(name='Drug related incidents', x=drug_halls_stats['Location'], y=drug_halls_stats['Incidents per 100 students'], marker_color='#548f6f'),
    go.Bar(name='Alcohol related incidents', x=alcohol_halls_stats['Location'], y=alcohol_halls_stats['Incidents per 100 students'],      marker_color='#f28e1c')
])
fig.update_layout(
    title={
        'text': "Number of Drug & Alcohol Related Incidents per 100 Residents by Residence Hall for the 2018-19 school year",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    font=dict(
        family="Helvetica",
        size=24
    ),
    xaxis_title='Residence Hall',
    yaxis_title='Total incidents reported',
    barmode='group',
    width=1920,
    height=1080,
    legend=dict(
        yanchor="top",
        y=0.96,
        xanchor="left",
        x=0.8
))
fig.write_image("images/fig4.png")